In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation="relu"),
        layers.Dense(embed_dim),]
 )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
    })
    return config

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
  keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
  save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
callbacks=callbacks)


In [ ]:
model = keras.models.load_model(
 "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})

# when using a cudtom layer always specify it when loading the model

## Machine Translation with Transformers Encoders

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Create a directory in Google Drive to store the dataset
!mkdir -p /content/drive/MyDrive/datasets
%cd /content/drive/MyDrive/datasets

# Step 3: Download the dataset using wget
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

# Step 4: Unzip the downloaded file
!unzip -q spa-eng.zip


Mounted at /content/drive
/content/drive/MyDrive/datasets
--2025-05-14 17:35:44--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.180.207, 142.251.179.207, 142.251.16.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.180.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip.1’

spa-eng.zip.1       100%[===================>]   2.52M  --.-KB/s    in 0.05s   

2025-05-14 17:35:44 (54.1 MB/s) - ‘spa-eng.zip.1’ saved [2638744/2638744]

replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace spa-eng/spa.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
text_file = "/content/drive/MyDrive/datasets/spa-eng/spa.txt"
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]
text_pairs = []

for line in lines:
  english, spanish = line.split("\t")
  spanish = "[start] " + spanish + " [end]"
  text_pairs.append((english, spanish))

In [ ]:
import random
print(random.choice(text_pairs))

('Prices are higher here than in Australia.', '[start] Los precios son más altos aquí que en Australia. [end]')


In [ ]:
 import random
 random.shuffle(text_pairs)
 num_val_samples = int(0.15 * len(text_pairs))
 num_train_samples = len(text_pairs) - 2 * num_val_samples


 train_pairs = text_pairs[:num_train_samples]
 val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
 test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [ ]:
# Vectorizing the English and Spanish text pairs
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
    lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length,
)

target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]

source_vectorization.adapt(train_english_texts)   # this adapt the vecorization to the train_english_text
target_vectorization.adapt(train_spanish_texts)

In [ ]:
batch_size = 64
def format_dataset(eng, spa):
  eng = source_vectorization(eng) # this applies the vectorization to the train ds
  spa = target_vectorization(spa)
  return ({
    "english": eng,
    "spanish": spa[:, :-1],   # the first part ie the dictionary is the inputs while
  }, spa[:, 1:])  # this last part is the traget as rnn tries to predict the next word in the sequence

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs) # this returns a tuple after spliting the pairs(the pairs were defined above)
    eng_texts = list(eng_texts) # truns the tuple to list
    spa_texts = list(spa_texts)

    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts)) # creates a tf.data.Dataset object with the eng and spa pair as input and target
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4) # applies the format_dataset function which further split the dataset object and set the a
                                                                # last word in the spanish part to be the output

    return dataset.shuffle(2048).prefetch(16).cache() # shuffles, prefetch to aid easier processing, cache to save to memory


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

## Sequence to Sequence ML translation with RNN

In [ ]:
# the encoder using GRU rnn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024
source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
  layers.GRU(latent_dim), merge_mode="sum")(x)

In [ ]:
# GRU based Decoder

past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x1 = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target) # Ensures embedding on past targets
decoder_gru = layers.GRU(latent_dim, return_sequences=True)

x = decoder_gru(x1, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)



In [ ]:
seq2seq_rnn.compile(
  optimizer="rmsprop",
  loss="sparse_categorical_crossentropy",
  metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)